In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
from datetime import datetime
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from dataloader.builder import build_dataset

from model.mlp import MLP
from model.ensemble import MLPEnsemble
from uncertainty_estimator.masks import build_masks, DEFAULT_MASKS
from experiment_setup import build_estimator, get_model
from analysis.metrics import get_uq_metrics
from experiments.utils.data import scale, split_ood, multiple_kfold
import torch

from analysis.autoencoder import VAE

from loss.nll_loss import NLLRegLoss
from torch.nn import functional as F

def nll_postprocessing(x):
    return torch.cat([x[:, [0]], F.softplus(x[:, [1]])], dim=1)

plt.rcParams['figure.facecolor'] = 'white'
torch.cuda.set_device(1)


config = {
    'nn_runs': 100,
    'runs': 2,
    'max_runs': 20,
    'k_folds': 10,
    'verbose': False,
    'layers': [8, 256, 256, 128, 1],
    'epochs': 10_000,
    'validation_step': 50,
    'acc_percentile': 0.1,
    'patience': 3,
    'dropout_rate': 0.2,
    'dropout_uq': 0.5,
    'batch_size': 256,
    'dataset': 'concrete',
    'ood_percentile': 90,
    'with_ensembles': True,
    'optimizer': {'type': 'Adadelta', 'weight_decay':1e-3}
}


In [ ]:

# Load dataset
dataset = build_dataset(config['dataset'])

x_train, y_train = dataset.dataset('train')
x_val, y_val = dataset.dataset('val')


In [ ]:
autoencoder = VAE(8, 64, 2)
autoencoder.train()
autoencoder.fit(x_train)


In [ ]:
np.unique(x_train).shape

In [ ]:
autoencoder.eval()
def encode(x):
    samples = torch.DoubleTensor(x).to('cuda')
    encoded, _ = autoencoder.encode(samples)
    return encoded.cpu().detach().numpy()
encoded_train = encode(x_train)
encoded_val = encode(x_val)

In [ ]:

sns.scatterplot(
    x=encoded_train[:, 0], y=encoded_train[:, 1], hue=y_train.squeeze(),
    # palette=sns.color_palette("Blues", 1282))
    palette="Blues")
sns.scatterplot(
    x=encoded_val[:, 0], y=encoded_val[:, 1], hue=y_val.squeeze(),
    # palette=sns.color_palette("Blues", 1282))
    palette="Reds")


In [ ]:
# Train or load model
model = MLP(config['layers'], optimizer=config['optimizer'])
model_path = f"experiments/data/model_{config['dataset']}.ckpt"
model = get_model(model, model_path, (x_train, y_train), (x_val, y_val))

# UE
masks = build_masks(['vanilla', 'rank_dpp', 'dpp'])
estimator = build_estimator('mcdue_masked', model, dropout_mask=masks['dpp'])
estimations = estimator.estimate(x_val)
print(estimations)
print(estimations.shape)





In [ ]:
import matplotlib.tri as tri
import numpy as np
from scipy.interpolate import griddata

np.random.seed(19680801)
npts = 200
ngridx = 200
ngridy = 200
# x = np.random.uniform(-2, 2, npts)
# y = np.random.uniform(-2, 2, npts)
# z = x * np.exp(-x**2 - y**2)
x = encoded_val[:, 0]
y = encoded_val[:, 1]
z = estimations

fig, ax1 = plt.subplots(figsize=(16, 9))

x1, x2 = 30, 120
y1, y2 = -80, 60
xi = np.linspace(x1, x2, ngridx)
yi = np.linspace(y1, y2, ngridy)
zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='linear')


ax1.contour(xi, yi, zi, levels=14, linewidths=0.5, colors='k')
cntr1 = ax1.contourf(xi, yi, zi, levels=14, cmap="gray")

fig.colorbar(cntr1, ax=ax1)
ax1.plot(x, y, 'ko', ms=3, color='red', alpha=0.3)
ax1.set(xlim=(x1, x2), ylim=(y1, y2))
ax1.set_title('grid and contour (%d points, %d grid points)' %
              (npts, ngridx * ngridy))

